In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "6"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

2024-06-17 14:57:13.957743: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 14:57:14.014816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 14:57:14.974594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
if True:
    with open("splitted_rws.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
        X_training, X_val, X_extraction = X_training[:, 9930:14790], X_val[:, 9930:14790], X_extraction[:, 9930:14790]
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_rws_only.pic", "rb") as r:
        traces_rws_only = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_rws_only, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_rws.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_rws_2", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_rws_2.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_rws_2.keras")
except ValueError:
    pass

2024-06-17 14:57:36.375625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-17 14:57:36.416463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-17 14:57:36.416518: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-17 14:57:36.429495: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-17 14:57:36.429570: I external/local_xla/xla/stream_executor

In [6]:
X_training = np.pad(X_training, ((0, 0), (resnet.model.input.shape[1] - X_training.shape[1], 0)))
X_val = np.pad(X_val, ((0, 0), (resnet.model.input.shape[1] - X_val.shape[1], 0)))

In [7]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [8]:
new_y_train = {}
new_y_val = {}
for output in train_gen.y.keys():
    idx = int(output[len('rws_mask_'):][:2])
    shx = int(output[len('rws_mask_XX_')])
    new_y_train[f'rws_mask_{idx - 48}_{shx}_output'] = train_gen.y[output]
    new_y_val[f'rws_mask_{idx - 48}_{shx}_output'] = val_gen.y[output]
train_gen.y = new_y_train
val_gen.y = new_y_val

In [9]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_rws_2.keras", patience=10)

Epoch 1/1000


2024-06-17 15:00:10.155579: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
/root/Pierugo/protected_side_channel_attack/.venv_linux/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718629210.642022   59829 service.cc:145] XLA service 0x7f452411f100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718629210.642074   59829 service.cc:153]   StreamExecutor device (0): NVIDIA RTX 4500 Ada Generation, Compute Capability 8.9
2024-06-17 15:00:12.359172: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc

 6851/10547 ━━━━━━━━━━━━━━━━━━━━ 6:38 108ms/step - loss: 130.3701 - rws_mask_0_0_output_accuracy: 0.1070 - rws_mask_0_1_output_accuracy: 0.0917 - rws_mask_10_0_output_accuracy: 0.1078 - rws_mask_10_1_output_accuracy: 0.0911 - rws_mask_11_0_output_accuracy: 0.1077 - rws_mask_11_1_output_accuracy: 0.0914 - rws_mask_12_0_output_accuracy: 0.1051 - rws_mask_12_1_output_accuracy: 0.0927 - rws_mask_13_0_output_accuracy: 0.1082 - rws_mask_13_1_output_accuracy: 0.0926 - rws_mask_14_0_output_accuracy: 0.1064 - rws_mask_14_1_output_accuracy: 0.0916 - rws_mask_15_0_output_accuracy: 0.1066 - rws_mask_15_1_output_accuracy: 0.0920 - rws_mask_16_0_output_accuracy: 0.1087 - rws_mask_16_1_output_accuracy: 0.0913 - rws_mask_17_0_output_accuracy: 0.1076 - rws_mask_17_1_output_accuracy: 0.0916 - rws_mask_18_0_output_accuracy: 0.1079 - rws_mask_18_1_output_accuracy: 0.0923 - rws_mask_19_0_output_accuracy: 0.1078 - rws_mask_19_1_output_accuracy: 0.0919 - rws_mask_1_0_output_accuracy: 0.1086 - rws_mask_1_1_ou

I0000 00:00:1718630033.652427   59835 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_69', 72 bytes spill stores, 72 bytes spill loads



10547/10547 ━━━━━━━━━━━━━━━━━━━━ 1302s 116ms/step - loss: 130.3694 - rws_mask_0_0_output_accuracy: 0.1071 - rws_mask_0_1_output_accuracy: 0.0916 - rws_mask_10_0_output_accuracy: 0.1077 - rws_mask_10_1_output_accuracy: 0.0914 - rws_mask_11_0_output_accuracy: 0.1078 - rws_mask_11_1_output_accuracy: 0.0915 - rws_mask_12_0_output_accuracy: 0.1055 - rws_mask_12_1_output_accuracy: 0.0925 - rws_mask_13_0_output_accuracy: 0.1082 - rws_mask_13_1_output_accuracy: 0.0923 - rws_mask_14_0_output_accuracy: 0.1064 - rws_mask_14_1_output_accuracy: 0.0915 - rws_mask_15_0_output_accuracy: 0.1065 - rws_mask_15_1_output_accuracy: 0.0920 - rws_mask_16_0_output_accuracy: 0.1086 - rws_mask_16_1_output_accuracy: 0.0914 - rws_mask_17_0_output_accuracy: 0.1074 - rws_mask_17_1_output_accuracy: 0.0917 - rws_mask_18_0_output_accuracy: 0.1077 - rws_mask_18_1_output_accuracy: 0.0923 - rws_mask_19_0_output_accuracy: 0.1076 - rws_mask_19_1_output_accuracy: 0.0919 - rws_mask_1_0_output_accuracy: 0.1083 - rws_mask_1_1_o